In [6]:
#朴素贝叶斯：要求特征之间相互独立
#https://blog.csdn.net/c406495762/column/info/16415
# -*- coding: UTF-8 -*-
import numpy as np
from functools import reduce

"""
函数说明:创建实验样本
Parameters:
    无
Returns:
    postingList - 实验样本切分的词条
    classVec - 类别标签向量
Author:
    Jack Cui
Blog:
    http://blog.csdn.net/c406495762
Modify:
    2017-08-11
"""
def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],#切分的词条
                ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1] #类别标签向量，1代表侮辱性词汇，0代表不是
    return postingList,classVec#返回实验样本切分的词条和类别标签向量

"""
函数说明:将切分的实验样本词条整理成不重复的词条列表，也就是词汇表
Parameters:
    dataSet - 整理的样本数据集
Returns:
    vocabSet - 返回不重复的词条列表，也就是词汇表
Author:
    Jack Cui
Blog:
    http://blog.csdn.net/c406495762
Modify:
    2017-08-11
"""
def createVocabList(dataSet):
    vocabSet = set([]) #创建一个空的不重复列表
    for document in dataSet:
        vocabSet = vocabSet | set(document) #取并集,相当于在set中add
#         print("document",document)
#         print("vocabSet",vocabSet)
    return list(vocabSet)

"""
函数说明:根据vocabList词汇表，将inputSet向量化，向量的每个元素为1或0，转化为伯努利分布
Parameters:
    vocabList - createVocabList返回的列表
    inputSet - 切分的词条列表
Returns:
    returnVec - 文档向量,词集模型
Author:
    Jack Cui
Blog:
    http://blog.csdn.net/c406495762
Modify:
    2017-08-11
"""
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList)#创建一个其中所含元素都为0的向量
    for word in inputSet:#遍历每个词条
        if word in vocabList:#如果词条存在于词汇表中，则置1
            returnVec[vocabList.index(word)] = 1
        else: print("the word: %s is not in my Vocabulary!" % word)
    return returnVec#返回文档向量


"""
函数说明:朴素贝叶斯分类器训练函数
Parameters:
    trainMatrix - 训练文档矩阵，转换为0-1矩阵，即setOfWords2Vec返回的returnVec构成的矩阵
    trainCategory - 训练类别标签向量，即loadDataSet返回的classVec
Returns:
    p0Vect - 非的条件概率数组
    p1Vect - 侮辱类的条件概率数组
    pAbusive - 文档属于侮辱类的概率
Author:
    Jack Cui
Blog:
    http://blog.csdn.net/c406495762
Modify:
    2017-08-12
"""
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)#计算训练的文档数目：6
    numWords = len(trainMatrix[0])#计算每篇文档的词条数：32
    pAbusive = sum(trainCategory)/float(numTrainDocs)#文档属于侮辱类的概率
    p0Num = np.ones(numWords); p1Num = np.ones(numWords)#创建numpy.ones数组,使用拉普拉斯平滑，p0Num存放侮辱类的条件概率，p1Num存放非侮辱类的条件概率
    p0Denom = 2.0; p1Denom = 2.0                        #分母初始化为
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:#统计属于侮辱类的条件概率所需的数据，即P(w0|1),P(w1|1),P(w2|1)···
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])                  ## 该词条的总的词数目   这压样求得每个词条出现的概率 P(w1),P(w2), P(w3)...
        else:#统计属于非侮辱类的条件概率所需的数据，即P(w0|0),P(w1|0),P(w2|0)···
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = np.log(p1Num/p1Denom)#相除        
    p0Vect = np.log(p0Num/p0Denom)#采用对数，避免下溢出          
    return p0Vect,p1Vect,pAbusive#返回属于侮辱类的条件概率数组，属于非侮辱类的条件概率数组，文档属于侮辱类的概率

"""
函数说明:朴素贝叶斯分类器分类函数
Parameters:
    vec2Classify - 待分类的词条数组
    p0Vec - 非侮辱类的条件概率数组
    p1Vec -侮辱类的条件概率数组
    pClass1 - 文档属于侮辱类的概率
Returns:
    0 - 属于非侮辱类
    1 - 属于侮辱类
Author:
    Jack Cui
Blog:
    http://blog.csdn.net/c406495762
Modify:
    2017-08-12
"""
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify*p1Vec)+np.log(pClass1)
    
    p0 = sum(vec2Classify*p0Vec)+np.log(1-pClass1)
    if p1 > p0:
        return 1
    else: 
        return 0

"""
函数说明:测试朴素贝叶斯分类器
Parameters:
    无
Returns:
    无
Author:
    Jack Cui
Blog:
    http://blog.csdn.net/c406495762
Modify:
    2017-08-12
"""
def testingNB():
    listOPosts,listClasses = loadDataSet()#创建实验样本
    myVocabList = createVocabList(listOPosts)#创建词汇表
    trainMat=[]#用于存放向量化后的二维数组
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))#将实验样本向量化，每次向量化一行数据，append是[[],[]……],不展开
    p0V,p1V,pAb = trainNB0(np.array(trainMat),np.array(listClasses))#训练朴素贝叶斯分类器
    testEntry = ['love', 'my', 'dalmation']#测试样本1
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))#测试样本向量化
    if classifyNB(thisDoc,p0V,p1V,pAb):
        print(testEntry,'属于侮辱类')#执行分类并打印分类结果
    else:
        print(testEntry,'属于非侮辱类')#执行分类并打印分类结果
    testEntry = ['stupid', 'garbage']#测试样本2

    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))#测试样本向量化
    if classifyNB(thisDoc,p0V,p1V,pAb):
        print(testEntry,'属于侮辱类')#执行分类并打印分类结果
    else:
        print(testEntry,'属于非侮辱类')#执行分类并打印分类结果

if __name__ == '__main__':
    testingNB()

['love', 'my', 'dalmation'] 属于非侮辱类
['stupid', 'garbage'] 属于侮辱类


In [ ]:
#vocabSet = vocabSet | set(document) #取并集,相当于在set中add

# def setOfWords2Vec(vocabList, inputSet):
#     returnVec = [0] * len(vocabList)#创建一个其中所含元素都为0的向量
#     for word in inputSet:#遍历每个词条
#         if word in vocabList:#如果词条存在于词汇表中，则置1
#             returnVec[vocabList.index(word)] = 1
#         else: print("the word: %s is not in my Vocabulary!" % word)
#     return returnVec#返回文档向量